In [6]:
import tkinter as tk
from tkinter import filedialog, messagebox
import shutil
import subprocess
import os
from openpyxl import load_workbook
from openpyxl.styles import Border, Side, PatternFill
from openpyxl.worksheet.properties import WorksheetProperties, PageSetupProperties
from openpyxl.formatting import Rule
from openpyxl.styles.differential import DifferentialStyle

#========================================================================================================================================================================
#For Trial and Rollout

def process_judging_trial(file_path):
    sheet_name = "Process judging document"
    output_file = "Process Judging Information_complete.xlsx"

    wb = load_workbook(file_path)
    ws = wb[sheet_name]
    merged_ranges = ws.merged_cells.ranges

    #Convert text to number
    columns_to_convert = [20, 24, 28, 36, 45] #T, X, AB, AJ, AS

    for col in columns_to_convert:
        for row in range(13, ws.max_row + 1):
            cell = ws.cell(row = row, column = col)
            try:
                if isinstance(cell.value, str) and cell.value.strip() != "":
                    num = float(cell.value.strip())
                    cell.value = num
            except ValueError:
                continue
    
    #Get value from top left of merged cell
    def get_top_left_value(row, col, merged_ranges):
        cell = ws.cell(row=row, column=col)
        for merge_range in merged_ranges:
            if cell.coordinate in merge_range:
                return ws.cell(merge_range.min_row, merge_range.min_col).value
        return cell.value
    
    #Get coordinate of merged cell
    def get_top_left_coord(row, col, merged_ranges):
        cell = ws.cell(row=row, column=col)
        for merge_range in merged_ranges:
            if cell.coordinate in merge_range:
                return merge_range.min_row, merge_range.min_col
        return row, col
    
    #Convert format to number
    def convert_to_number(value):
        try:
            return float(str(value).strip())
        except (TypeError, ValueError):
            return None

    # Load keep value
    reference_file_path = os.path.join("Require", "For reference value.xlsx")
    wb_ref = load_workbook(reference_file_path, data_only=True)
    ws_ref = wb_ref["Keep"]
    keep_values = set()
    for row in ws_ref.iter_rows(min_row=2, max_col=1, values_only=True):
        if row[0] is not None:
            keep_values.add(str(row[0]).strip())
    
    # Delete O or blank except column O match value in keep_values
    rows_to_delete = set()
    for row in range(13, ws.max_row + 1):
        col_o_value = get_top_left_value(row, 15, merged_ranges)
        col_aw_value = get_top_left_value(row, 49, merged_ranges)
    
        # if column O match value in keep >> skip
        if str(col_o_value).strip() in keep_values:
            continue

        # id column O not match value in keep >> go executing process
        if col_aw_value is None or str(col_aw_value).strip() == "" or col_aw_value == "O":
            rows_to_delete.add(row)
    
    for row in sorted(rows_to_delete, reverse=True):
        ws.delete_rows(row)

    #Calculate value in "Deviet" column
    col_bk = 64 #Deviet column
    col_aj = 36 #Eval value column
    col_t = 20 #Cent value column   
    for row in range(13, ws.max_row + 1):
        value_aj = convert_to_number(get_top_left_value(row, col_aj, merged_ranges))
        value_t = convert_to_number(get_top_left_value(row, col_t, merged_ranges))    
        result = f'=round(AJ{row} - T{row}, 1)'

        write_row, write_col = get_top_left_coord(row, col_bk, merged_ranges)
        ws.cell(row = write_row, column = write_col).value = result

    #Set "Compensation coefficient" equal to 1
    col_bo = 68 #Compensation coef column   
    for row in range(13, ws.max_row + 1):
        result = 1 
        write_row, write_col = get_top_left_coord(row, col_bo, merged_ranges)
        ws.cell(row = write_row, column = write_col).value = result

    #Calculate "Fiducial point" range
    col_bw = 76 #Fiducial point column
    col_t = 20 #Cent value column
    col_x = 24 #Max column
    col_ab = 28 #Min column
    col_as = 45 #Pass std column
    for row in range(13, ws.max_row + 1):
        value_t = convert_to_number(get_top_left_value(row, col_t, merged_ranges))
        value_x = convert_to_number(get_top_left_value(row, col_x, merged_ranges))
        value_ab = convert_to_number(get_top_left_value(row, col_ab, merged_ranges))
        value_as = convert_to_number(get_top_left_value(row, col_as, merged_ranges))
        
        if value_t is None and value_ab is None and value_x is not None:
            result = f'="Max " & round(X{row}, 1)'
        elif value_t is None and value_ab is not None and value_x is None:
            result = f'="Min " & round(AB{row}, 1)'
        elif value_t is None and value_ab is not None and value_x is not None:
            result = f'=round(AB{row}, 1) & " - " & round(X{row}, 1)'
        elif value_t is not None and value_ab is not None and value_x is None:
            result = f'="Min " & round(AB{row}, 1)'
        elif value_t is not None and value_ab is None and value_x is not None:
            result = f'="Max " & round(X{row}, 1)'
        else:
            result = f'=round(T{row} - AS{row}, 1) & " - " & round(T{row} + AS{row}, 1)'
            
        write_row, write_col = get_top_left_coord(row, col_bw, merged_ranges)
        ws.cell(row = write_row, column = write_col).value = result

    #Value "For reference"
    reference_wb = load_workbook(reference_file_path)
    reference_ws = reference_wb["Reference"]   
    reference_keys = set()
    for row in range(2, reference_ws.max_row + 1):
        key = reference_ws.cell(row = row, column = 1).value
        reference_keys.add(key)
    for row in range(13, ws.max_row + 1):
        key = ws.cell(row = row, column = 15).value
        if key in reference_keys:
            write_row, write_col = get_top_left_coord(row, 82, merged_ranges)
            ws.cell(row = write_row, column = write_col).value = "For reference"

    #Calculate "Compensation value"
    col_bg = 60 #Actual point
    col_bs = 72 #Compensation value
    col_aj = 36 #Eval value
    col_bo = 68 #Compensation coeficient
    for  row in range(13, ws.max_row + 1):
        result = f'=AJ{row} - (BG{row} * BO{row})'
        write_row, write_col = get_top_left_coord(row, col_bs, merged_ranges)
        ws.cell(row = write_row, column = write_col).value = result

    #Summary result of "Process Judging"
    col_ca = 80 #Process judging
    col_bs = 72 #Compensation value
    col_t = 20 #Cent value
    col_x = 24 #Max column
    col_ab = 28 #Min column
    col_as = 45 #Pass std column
    for row in range(13, ws.max_row + 1):
        value_t = convert_to_number(get_top_left_value(row, col_t, merged_ranges))
        value_x = convert_to_number(get_top_left_value(row, col_x, merged_ranges))
        value_ab = convert_to_number(get_top_left_value(row, col_ab, merged_ranges))
        value_as = convert_to_number(get_top_left_value(row, col_as, merged_ranges))
        value_bs = convert_to_number(get_top_left_value(row, col_bs, merged_ranges))
        value_ca = convert_to_number(get_top_left_value(row, col_ca, merged_ranges))
        
        if value_t is None and value_ab is None and value_x is not None:
            result = f'=IFERROR(IF(BS{row}<=X{row}, "OK", "NG"), "")'
        elif value_t is None and value_ab is not None and value_x is None:
            result = f'=IFERROR(IF(BS{row}>=AB{row}, "OK", "NG"), "")'
        elif value_t is None and value_ab is not None and value_x is not None:
            result = f'=IFERROR(IF(AND(BS{row}<=X{row}, BS{row}>=AB{row}), "OK", "NG"), "")'
        elif value_t is not None and value_ab is not None and value_x is None:
            result = f'=IFERROR(IF(BS{row}>=AB{row}, "OK", "NG"), "")'
        elif value_t is not None and value_ab is None and value_x is not None:
            result = f'=IFERROR(IF(BS{row}<=X{row}, "OK", "NG"), "")'           
        else:
            result = f'=IFERROR(IF(AND(BS{row}>=T{row}-AS{row}, BS{row}<=T{row}+AS{row}), "OK", "NG"), "")'

        write_row, write_col = get_top_left_coord(row, col_ca, merged_ranges)
        ws.cell(row = write_row, column = write_col).value = result

    #Conditional formatting
    highlight_fill = PatternFill(start_color = "FDB4A5", end_color = "FDB4A5", fill_type = "solid")
    dxf = DifferentialStyle(fill = highlight_fill)

    formula = '$CA13="NG"'

    rule = Rule(type = "expression", dxf = dxf, formula = [formula])

    target_range = f"B13:CL{ws.max_row}"
    ws.conditional_formatting.add(target_range, rule)
    
    #Add remark and dot line
    last_row = ws.max_row
    ws.cell(row=last_row + 2, column=2).value = "Remark"

    dotted_border = Border(bottom=Side(style="dotted"))
    for col in range(2, 91):  # B ถึง CL (column 2 ถึง 90)
        cell = ws.cell(row=last_row + 3, column=col)
        cell2 = ws.cell(row=last_row + 4, column=col)
        cell.border = dotted_border
        cell2.border = dotted_border

    #Print page setup
    ws.sheet_properties = WorksheetProperties(
        pageSetUpPr=PageSetupProperties(fitToPage=True)
    )
    
    ws.page_setup.fitToWidth = 1        
    ws.page_setup.fitToHeight = 0       
    ws.page_setup.paperSize = ws.PAPERSIZE_A4
    ws.page_setup.orientation = "landscape"  
    
    ws.print_options.horizontalCentered = True


    #Save file
    wb.save(os.path.join("Require", output_file))

#========================================================================================================================================================================
#For Initial and Mass control

def process_judging_initial(file_path):
    sheet_name = "Process judging document"
    output_file = "Process Judging Information_complete.xlsx"

    wb = load_workbook(file_path)
    ws = wb[sheet_name]
    merged_ranges = ws.merged_cells.ranges

    #Convert text to number
    columns_to_convert = [20, 24, 28, 36, 45] #T, X, AB, AJ, AS

    for col in columns_to_convert:
        for row in range(13, ws.max_row + 1):
            cell = ws.cell(row = row, column = col)
            try:
                if isinstance(cell.value, str) and cell.value.strip() != "":
                    num = float(cell.value.strip())
                    cell.value = num
            except ValueError:
                continue
    
    #Get value from top left of merged cell
    def get_top_left_value(row, col, merged_ranges):
        cell = ws.cell(row=row, column=col)
        for merge_range in merged_ranges:
            if cell.coordinate in merge_range:
                return ws.cell(merge_range.min_row, merge_range.min_col).value
        return cell.value
    
    #Get coordinate of merged cell
    def get_top_left_coord(row, col, merged_ranges):
        cell = ws.cell(row=row, column=col)
        for merge_range in merged_ranges:
            if cell.coordinate in merge_range:
                return merge_range.min_row, merge_range.min_col
        return row, col
    
    #Convert format to number
    def convert_to_number(value):
        try:
            return float(str(value).strip())
        except (TypeError, ValueError):
            return None
    
    # Delete O or blank 
    reference_file_path = os.path.join("Require", "For reference value.xlsx")
    rows_to_delete = set()
    for row in range(13, ws.max_row + 1):
        col_aw_value = get_top_left_value(row, 49, merged_ranges)

        # Executing process
        if col_aw_value is None or str(col_aw_value).strip() == "" or col_aw_value == "O":
            rows_to_delete.add(row)
    
    for row in sorted(rows_to_delete, reverse=True):
        ws.delete_rows(row)

    #Calculate value in "Deviet" column
    col_bk = 64 #Deviet column
    col_aj = 36 #Eval value column
    col_t = 20 #Cent value column   
    for row in range(13, ws.max_row + 1):
        value_aj = convert_to_number(get_top_left_value(row, col_aj, merged_ranges))
        value_t = convert_to_number(get_top_left_value(row, col_t, merged_ranges))    
        result = f'=round(AJ{row} - T{row}, 1)'

        write_row, write_col = get_top_left_coord(row, col_bk, merged_ranges)
        ws.cell(row = write_row, column = write_col).value = result

    #Set "Compensation coefficient" equal to 1
    col_bo = 68 #Compensation coef column   
    for row in range(13, ws.max_row + 1):
        result = 1 
        write_row, write_col = get_top_left_coord(row, col_bo, merged_ranges)
        ws.cell(row = write_row, column = write_col).value = result

    #Calculate "Fiducial point" range
    col_bw = 76 #Fiducial point column
    col_t = 20 #Cent value column
    col_x = 24 #Max column
    col_ab = 28 #Min column
    col_as = 45 #Pass std column
    for row in range(13, ws.max_row + 1):
        value_t = convert_to_number(get_top_left_value(row, col_t, merged_ranges))
        value_x = convert_to_number(get_top_left_value(row, col_x, merged_ranges))
        value_ab = convert_to_number(get_top_left_value(row, col_ab, merged_ranges))
        value_as = convert_to_number(get_top_left_value(row, col_as, merged_ranges))
        
        if value_t is None and value_ab is None and value_x is not None:
            result = f'="Max " & round(X{row}, 1)'
        elif value_t is None and value_ab is not None and value_x is None:
            result = f'="Min " & round(AB{row}, 1)'
        elif value_t is None and value_ab is not None and value_x is not None:
            result = f'=round(AB{row}, 1) & " - " & round(X{row}, 1)'
        elif value_t is not None and value_ab is not None and value_x is None:
            result = f'="Min " & round(AB{row}, 1)'
        elif value_t is not None and value_ab is None and value_x is not None:
            result = f'="Max " & round(X{row}, 1)'
        else:
            result = f'=round(T{row} - AS{row}, 1) & " - " & round(T{row} + AS{row}, 1)'
            
        write_row, write_col = get_top_left_coord(row, col_bw, merged_ranges)
        ws.cell(row = write_row, column = write_col).value = result

    #Value "For reference"
    reference_wb = load_workbook(reference_file_path)
    reference_ws = reference_wb["Reference"]   
    reference_keys = set()
    for row in range(2, reference_ws.max_row + 1):
        key = reference_ws.cell(row = row, column = 1).value
        reference_keys.add(key)
    for row in range(13, ws.max_row + 1):
        key = ws.cell(row = row, column = 15).value
        if key in reference_keys:
            write_row, write_col = get_top_left_coord(row, 82, merged_ranges)
            ws.cell(row = write_row, column = write_col).value = "For reference"

    #Calculate "Compensation value"
    col_bg = 60 #Actual point
    col_bs = 72 #Compensation value
    col_aj = 36 #Eval value
    col_bo = 68 #Compensation coeficient
    for  row in range(13, ws.max_row + 1):
        result = f'=AJ{row} - (BG{row} * BO{row})'
        write_row, write_col = get_top_left_coord(row, col_bs, merged_ranges)
        ws.cell(row = write_row, column = write_col).value = result

    #Summary result of "Process Judging"
    col_ca = 80 #Process judging
    col_bs = 72 #Compensation value
    col_t = 20 #Cent value
    col_x = 24 #Max column
    col_ab = 28 #Min column
    col_as = 45 #Pass std column
    for row in range(13, ws.max_row + 1):
        value_t = convert_to_number(get_top_left_value(row, col_t, merged_ranges))
        value_x = convert_to_number(get_top_left_value(row, col_x, merged_ranges))
        value_ab = convert_to_number(get_top_left_value(row, col_ab, merged_ranges))
        value_as = convert_to_number(get_top_left_value(row, col_as, merged_ranges))
        value_bs = convert_to_number(get_top_left_value(row, col_bs, merged_ranges))
        value_ca = convert_to_number(get_top_left_value(row, col_ca, merged_ranges))
        
        if value_t is None and value_ab is None and value_x is not None:
            result = f'=IFERROR(IF(BS{row}<=X{row}, "OK", "NG"), "")'
        elif value_t is None and value_ab is not None and value_x is None:
            result = f'=IFERROR(IF(BS{row}>=AB{row}, "OK", "NG"), "")'
        elif value_t is None and value_ab is not None and value_x is not None:
            result = f'=IFERROR(IF(AND(BS{row}<=X{row}, BS{row}>=AB{row}), "OK", "NG"), "")'
        elif value_t is not None and value_ab is not None and value_x is None:
            result = f'=IFERROR(IF(BS{row}>=AB{row}, "OK", "NG"), "")'
        elif value_t is not None and value_ab is None and value_x is not None:
            result = f'=IFERROR(IF(BS{row}<=X{row}, "OK", "NG"), "")'           
        else:
            result = f'=IFERROR(IF(AND(BS{row}>=T{row}-AS{row}, BS{row}<=T{row}+AS{row}), "OK", "NG"), "")'

        write_row, write_col = get_top_left_coord(row, col_ca, merged_ranges)
        ws.cell(row = write_row, column = write_col).value = result

    #Conditional formatting
    highlight_fill = PatternFill(start_color = "FDB4A5", end_color = "FDB4A5", fill_type = "solid")
    dxf = DifferentialStyle(fill = highlight_fill)

    formula = '$CA13="NG"'

    rule = Rule(type = "expression", dxf = dxf, formula = [formula])

    target_range = f"B13:CL{ws.max_row}"
    ws.conditional_formatting.add(target_range, rule)
    
    #Add remark and dot line
    last_row = ws.max_row
    ws.cell(row=last_row + 2, column=2).value = "Remark"

    dotted_border = Border(bottom=Side(style="dotted"))
    for col in range(2, 91):  # B ถึง CL (column 2 ถึง 90)
        cell = ws.cell(row=last_row + 3, column=col)
        cell2 = ws.cell(row=last_row + 4, column=col)
        cell.border = dotted_border
        cell2.border = dotted_border

    #Print page setup
    ws.sheet_properties = WorksheetProperties(
        pageSetUpPr=PageSetupProperties(fitToPage=True)
    )
    
    ws.page_setup.fitToWidth = 1        
    ws.page_setup.fitToHeight = 0       
    ws.page_setup.paperSize = ws.PAPERSIZE_A4
    ws.page_setup.orientation = "landscape"  
    
    ws.print_options.horizontalCentered = True


    #Save file
    wb.save(os.path.join("Require", output_file))

#========================================================================================================================================================================

def run_gui():
    root = tk.Tk()
    root.title("Process Judging Information")
    root.geometry("400x200")
    root.iconbitmap(os.path.join("Require", "BridgestoneICO.ico"))

    def run_processing(mode):
        try:
            excel_file = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx")])
            if not excel_file:
                return
            
            if mode == "trial":
                process_judging_trial(excel_file)
            elif mode == "initial":
                process_judging_initial(excel_file)

            output_path = filedialog.asksaveasfilename(
                defaultextension=".xlsx",
                initialfile="Process Judging Information_complete.xlsx"
            )
            if output_path:
                source_path = os.path.join("Require", "Process Judging Information_complete.xlsx")
                shutil.copy(source_path, output_path)
                messagebox.showinfo("Done!", f"Your file has been saved at:\n{output_path}")
                os.remove(source_path)

        except Exception as e:
            messagebox.showerror("Error", str(e))

    label = tk.Label(root, text="Process Judgement", font=("TH SarabunPSK", 14))
    label.pack(pady=20)

    button_frame = tk.Frame(root)
    button_frame.pack(pady=10)

    button_trial = tk.Button(button_frame, text="Trial/Rollout", font=("TH SarabunPSK", 14),
                             command=lambda: run_processing("trial"), width=15)
    button_trial.pack(side="left", padx=10)

    button_initial = tk.Button(button_frame, text="Initial/Mass", font=("TH SarabunPSK", 14),
                               command=lambda: run_processing("initial"), width=15)
    button_initial.pack(side="right", padx=10)

    root.mainloop()


#====================================================================================

# GUI
if __name__ == "__main__":
    run_gui()